In [1]:
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
#import datetime as dt
from datetime import datetime as dt
import datetime
from datetime import date
import teradata
#import getpass
import time
import calendar
from mlutils import dataset
from mlutils import connector
start_time = time.time()

## Clean and keep only the dependencies you need!

### Set the time

In [2]:
dt_start = '2024-01-27'
monthend = str(date.today())

In [3]:
##tns_claim_table
#connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claim_table""") tns_claims_items_orders_1

In [4]:
query="select item.claim_id, claim.claim_type_rpt_nm, \
 claim.claim_type_actl_nm,claim.claim_brand_nm, \
 claim.claim_intellectual_prop_ownr_nm, claimant.claimant_email_addr_txt, \
 claim.claim_slr_nm_array, item.catlg_item_id, \
 claim.claim_curr_status_nm, offr.offr_actn_nm, \
 claim.created_date, item.src_upd_ts,\
 offr.slr_legacy_org_id, item.item_actn_rsn_nm,\
 offr.offr_catg_nm, offr.offr_pub_status_nm,\
 offr.dcsn_basis_nm, offr.offr_id,\
 claim.claim_desc_txt, claim.claim_cntry_txt\
         from `wmt-ww-gg-prod.ww_gg_gec_tns_sp_dl_secure.intellectual_prop_claim` claim \
 left join `wmt-ww-gg-prod.ww_gg_gec_tns_sp_dl_secure.intellectual_prop_claim_item` item \
             on (item.claim_id=claim.claim_id) \
 left join `wmt-ww-gg-prod.ww_gg_gec_tns_sp_dl_secure.intellectual_prop_claim_offr` offr \
             on (claim.claim_id=offr.claim_id and item.catlg_item_id = offr.catlg_item_id) \
 left join `wmt-ww-gg-prod.ww_gg_gec_tns_sp_dl_secure.intellectual_prop_claimant` claimant \
             on (claim.claimant_id = claimant.claimant_id) \
         where claim.created_date >= '2024-01-27' \
         AND claim.claim_type_actl_nm like '%Counterfeit' \
         AND item.claim_rev_type_cd = 'IPRR'"
        
claim_table  = dataset.load(name="bq_connection", query = query)


## Why do you need to look at the data and data types in a script that runs by itself?

In [5]:
#claim_table = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claim_table""")
#claim_table.head()

In [6]:
#claim_table.groupby('created_date').count().tail(10)

In [7]:
#claim_table.info()

In [8]:
claim_table.shape

(6912, 20)

## It is a good practice to name all columns with lower case letters and no special characters. Spaces are replaced with underscore.

In [9]:
claim_table.columns = ['Claim ID', 'Reported Claim Type', 'Actual Claim Type', 'Brand',
       'IP Company Name', 'Email Address', 'Reported Sellers', 'Item ID','Claim State', 
       'Action', 'Created', 'Updated','Legacy Seller ID', 'Reason Code',
       'Category', 'Offer Publish Status',  'Decision Type', 'ID', 'Reported Details','Country']

In [10]:
print(f"Number of uniquue claims is {claim_table['ID'].nunique()}")

Number of uniquue claims is 5302


In [11]:
#claim_table['ID']

### All Claims

## Name dataframes and any other variables as descriptive as possible. These could be `closed_claims_df` and `blocked_claims_df` I guess.

In [12]:
claim_table1=claim_table[claim_table['Claim State'] == 'Closed']
claim_table1=claim_table1[claim_table1['Action'] == 'Block']

In [13]:
#claim_table1.info()

## Why do you have to replace NAs with blanks?

In [14]:
claim_table1.fillna("", inplace = True)

In [15]:
claim_table_new=claim_table[['Claim ID', 'Actual Claim Type', 'Brand', 'Item ID','Claim State', 'Action', 'Created', 'Reason Code', 'Category', 'Offer Publish Status', 'ID', ]]

## If there are duplicates you are dropping them completely here! Do you want to?

In [16]:
claim_table_new.drop_duplicates(keep=False, inplace=True)

/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [17]:
offerlist = claim_table1['ID'].unique()
new_offer_list_1 = ','.join("'{0}'".format(w) for w in offerlist)

In [18]:
offerlist.size

240

In [19]:
offerlist_new = claim_table['ID'].unique()
new_offer_list = ','.join("'{0}'".format(w) for w in offerlist_new)

In [20]:
offerlist_new.size

5302

## Please delete all commented lines of code. 

In [21]:
#tns_catalog_new3.to_csv('catalog_order' +str(date.today())+'_confirmed_only.txt',sep='|',index=False,header=False)
#filepath = 'catalog_order' +str(date.today())+'_confirmed_only.txt'
#connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_catalog_table_confirmed_only/" + filepath)

In [22]:
#connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claim_order""")

In [23]:
#tns_claim_order = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claim_order""")

In [24]:
#statement=" CREATE TABLE tnsanalytics.tns_claim_order as \
#select src_tenant_sku_id as offer_id, order_placed_dt,cust_id,brand_nm as brand_name, \
#rpt_lvl_1_orig_nm as category, \
#COUNT(DISTINCT sales_order_num) as Order_Number, \
#sum(gmv_order_amt) as Total_Revenue, sum(units) as Total_Quantity, \
#sum(gmv_plcd_amt) as Placed_Amount, sum(plcd_units) as Placed_Quantity, \
#sum(gmv_cncl_amt) as Total_cancel_Amount, sum(cncl_units) as Cancelled_Quantity, \
#sum(gmv_rfnd_amt) as Refunded_Amount, sum(rtn_units) as Return_Quantity \
#from ww_crew_dl_rpt_vm.cnsld_order_item \
#where order_placed_dt <= date'"+monthend+"' and order_placed_dt >=date'"+dt_start+"'\
#and src_tenant_sku_id in ("+new_offer_list_1+") \
#group by src_tenant_sku_id, order_placed_dt,cust_id,brand_nm,rpt_lvl_1_orig_nm "

#print(statement)

In [25]:
##claim orders from ww_crew_dl_rpt_vm.cnsld_order_item_orig##
"""
connector.execute(name="gcp_dd_connection", statement=" CREATE TABLE tnsanalytics.tns_claim_order as \
select src_tenant_sku_id as offer_id, order_placed_dt,cust_id, \
COUNT(DISTINCT sales_order_num) as Order_Number, \
sum(gmv_order_amt) as Total_Revenue, sum(units) as Total_Quantity, \
sum(gmv_plcd_amt) as Placed_Amount, sum(plcd_units) as Placed_Quantity, \
sum(gmv_cncl_amt) as Total_cancel_Amount, sum(cncl_units) as Cancelled_Quantity, \
sum(gmv_rfnd_amt) as Refunded_Amount, sum(rtn_units) as Return_Quantity \
from ww_crew_dl_rpt_vm.cnsld_order_item \
where order_placed_dt <= date'"+monthend+"' and order_placed_dt >=date'"+dt_start+"'\
and src_tenant_sku_id in ("+new_offer_list_1+" \
) group by src_tenant_sku_id, order_placed_dt,cust_id ")
"""

'\nconnector.execute(name="gcp_dd_connection", statement=" CREATE TABLE tnsanalytics.tns_claim_order as select src_tenant_sku_id as offer_id, order_placed_dt,cust_id, COUNT(DISTINCT sales_order_num) as Order_Number, sum(gmv_order_amt) as Total_Revenue, sum(units) as Total_Quantity, sum(gmv_plcd_amt) as Placed_Amount, sum(plcd_units) as Placed_Quantity, sum(gmv_cncl_amt) as Total_cancel_Amount, sum(cncl_units) as Cancelled_Quantity, sum(gmv_rfnd_amt) as Refunded_Amount, sum(rtn_units) as Return_Quantity from ww_crew_dl_rpt_vm.cnsld_order_item where order_placed_dt <= date\'"+monthend+"\' and order_placed_dt >=date\'"+dt_start+"\'and src_tenant_sku_id in ("+new_offer_list_1+" ) group by src_tenant_sku_id, order_placed_dt,cust_id ")\n'

In [26]:
#query ="select * from `wmt-edw-prod.WW_PRODUCT_DL_VM.CNSLD_TACTICAL_SKU` LIMIT 10"

#TNS = dataset.load(name="bq_connection", query = query)
#TNS.info()

In [27]:
#CREATE TABLE `wmt-tns-analytics-prod.arbitrage.ReturnHubs`
#CREATE TABLE `wmt-tns-analytics-prod.arbitrage.tns_claim_order` as \

statement="select offr_id as offer_id, order_plcd_dt,cust_id, \
COUNT(DISTINCT order_child_nbr) as Order_Number, \
sum(gmv_order_amt) as Total_Revenue, sum(net_cncl_qty) as Total_Quantity, \
sum(gmv_plcd_amt) as Placed_Amount, sum(plcd_qty) as Placed_Quantity, \
sum(gmv_cncl_amt) as Total_cancel_Amount, sum(cncl_qty) as Cancelled_Quantity, \
sum(gmv_rfnd_amt) as Refunded_Amount, sum(rtn_rfnd_full_qty) as Return_Quantity \
from `wmt-edw-prod.WW_CREW_DL_RPT_VM.CNSLD_ORDER_ITEM` \
where order_plcd_dt <= date'"+monthend+"' and order_plcd_dt >=date'"+dt_start+"'\
and offr_id in ("+new_offer_list_1+" \
) group by offr_id, order_plcd_dt,cust_id "


In [28]:

#statement2 = 'select * from `wmt-edw-prod.WW_CREW_DL_RPT_VM.CNSLD_ORDER_ITEM` limit 10'

#query ="select * from `wmt-edw-prod.WW_PRODUCT_DL_VM.CNSLD_TACTICAL_SKU` LIMIT 10"

tns_claim_order = dataset.load(name="bq_connection", query = statement)
#tns_claim_order.info()


In [29]:
statement="select offr_id as offer_id, order_plcd_dt, \
COUNT(DISTINCT order_child_nbr) as Order_Number, \
sum(gmv_order_amt) as Total_Revenue, sum(net_cncl_qty) as Total_Quantity, \
sum(gmv_plcd_amt) as Placed_Amount, sum(plcd_qty) as Placed_Quantity, \
sum(gmv_cncl_amt) as Total_cancel_Amount, sum(cncl_qty) as Cancelled_Quantity, \
sum(gmv_rfnd_amt) as Refunded_Amount, sum(rtn_rfnd_full_qty) as Return_Quantity \
from `wmt-edw-prod.WW_CREW_DL_RPT_VM.CNSLD_ORDER_ITEM` \
where order_plcd_dt <= date'"+monthend+"' and order_plcd_dt >=date'"+dt_start+"'\
and offr_id in ("+new_offer_list+" \
) group by offr_id, order_plcd_dt "


In [30]:
tns_claim_order_new = dataset.load(name="bq_connection", query = statement)
tns_claim_order_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28411 entries, 0 to 28410
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   offer_id             28411 non-null  object 
 1   order_plcd_dt        28411 non-null  object 
 2   Order_Number         28411 non-null  int64  
 3   Total_Revenue        28085 non-null  object 
 4   Total_Quantity       28411 non-null  int64  
 5   Placed_Amount        28085 non-null  object 
 6   Placed_Quantity      28411 non-null  int64  
 7   Total_cancel_Amount  28085 non-null  object 
 8   Cancelled_Quantity   7234 non-null   float64
 9   Refunded_Amount      28085 non-null  object 
 10  Return_Quantity      28411 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 2.4+ MB


In [31]:
tns_claim_order.groupby('order_plcd_dt').count().tail(10)

,offer_id,cust_id,Order_Number,Total_Revenue,Total_Quantity,Placed_Amount,Placed_Quantity,Total_cancel_Amount,Cancelled_Quantity,Refunded_Amount,Return_Quantity
order_plcd_dt,,,,,,,,,,,
2024-04-12,39,39,39,37,39,37,39,37,3,37,39
2024-04-13,20,20,20,18,20,18,20,18,3,18,20
2024-04-14,30,30,30,29,30,29,30,29,1,29,30
2024-04-15,19,19,19,17,19,17,19,17,3,17,19
2024-04-16,22,22,22,22,22,22,22,22,1,22,22
2024-04-17,11,11,11,11,11,11,11,11,0,11,11
2024-04-18,5,5,5,5,5,5,5,5,0,5,5
2024-04-19,15,15,15,15,15,15,15,15,0,15,15
2024-04-20,5,5,5,5,5,5,5,5,0,5,5


In [32]:
#tns_claim_order['gmv_cncl_amt']

In [33]:
#tns_claim_order = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claim_order""")

In [34]:
tns_claim_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7681 entries, 0 to 7680
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   offer_id             7681 non-null   object 
 1   order_plcd_dt        7681 non-null   object 
 2   cust_id              7681 non-null   object 
 3   Order_Number         7681 non-null   int64  
 4   Total_Revenue        7442 non-null   object 
 5   Total_Quantity       7681 non-null   int64  
 6   Placed_Amount        7442 non-null   object 
 7   Placed_Quantity      7681 non-null   int64  
 8   Total_cancel_Amount  7442 non-null   object 
 9   Cancelled_Quantity   495 non-null    float64
 10  Refunded_Amount      7442 non-null   object 
 11  Return_Quantity      7681 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 720.2+ KB


In [35]:
tns_claim_order.fillna(0, inplace=True)

In [36]:
Claim_df = claim_table.groupby('ID').agg({'Claim ID' : 'first', 'Created' :'first', 'Item ID' :'first', 'Actual Claim Type' : 'first'})
Claim_df = Claim_df.reset_index()

In [37]:
#Claim_df

In [38]:
Claim_df_new = claim_table_new.groupby('ID').agg({'Claim ID' : 'first', 'Created' :'first', 'Item ID' :'first', 'Actual Claim Type' : 'first', 'Claim State' : 'first', 'Action' : 'first', 'Reason Code' : 'first', 'Category' : 'first' })
Claim_df_new = Claim_df_new.reset_index()

In [39]:
Claim_df['ID'].nunique()

5302

In [40]:
Claim_df_new['ID'].nunique()

5302

In [41]:
Claim_df_new.columns

Index(['ID', 'Claim ID', 'Created', 'Item ID', 'Actual Claim Type',
       'Claim State', 'Action', 'Reason Code', 'Category'],
      dtype='object')

In [42]:
Claim_df.groupby('ID').count()

,Claim ID,Created,Item ID,Actual Claim Type
ID,,,,
,1,1,1,1
000FF4373C9133C79C236B66C157FCFE,1,1,1,1
001CE34B1A8E3ED4AC392872120B2268,1,1,1,1
001D3417D99340ACA4CFD81B49433FF4,1,1,1,1
0025348312F1336293B383C41A9A08E5,1,1,1,1
...,...,...,...,...
FFD81AE00BEE3F8898CC5C82AC0A62FB,1,1,1,1
FFE0FCD285F0422F98B9082961BE01F6,1,1,1,1
FFE38484E0804424A4335D06075F6153,1,1,1,1


In [43]:
cf_claims_and_order = pd.merge(Claim_df_new, tns_claim_order_new, left_on = 'ID', right_on = 'offer_id', how = 'left')

In [44]:
cf_claims_and_order['ID'].nunique()

5302

In [45]:
#cf_claims_and_order

In [46]:
#combine claim table and **** together to create claim_orders table#
new_df = pd.merge(tns_claim_order, Claim_df, left_on = 'offer_id', right_on = 'ID', how = 'left')

In [47]:
new_df.shape

(7681, 17)

In [48]:
new_df['Claim date']= pd.to_datetime(new_df['Created']).dt.date
new_df['Claim date'].value_counts().sort_index()

2024-01-29     58
2024-01-30     88
2024-02-01    104
2024-02-02      6
2024-02-05     82
             ... 
2024-04-15      2
2024-04-16      7
2024-04-17     17
2024-04-18     13
2024-04-19    331
Name: Claim date, Length: 61, dtype: int64

In [49]:
new_df['Claim date']= pd.to_datetime(new_df['Created']).dt.date
new_df['order_plcd_dt']= pd.to_datetime(new_df['order_plcd_dt']).dt.date
new_df['delta'] = (new_df['Claim date'] - new_df['order_plcd_dt']).dt.days

In [50]:
#new_df['delta'].value_counts()

In [51]:
#looking back 30 days based on (claim date-order_placed_dt)
new_df_1 = new_df[(new_df['delta'] >= 0) & (new_df['delta'] <= 30)]
#new_df_1 = new_df[(new_df['delta'] <= 30)]

In [52]:
#new_df_1.sort_values('order_plcd_dt')

In [53]:
#new_df_1.groupby('order_plcd_dt').count().tail(10)

In [54]:
#new_df_1.info()

In [55]:
new_df_1.columns = ['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number', 'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity', 'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount', 'Return_Quantity', 'ID', 'Item ID', 'Claim ID', 'Created', 'Actual Claim Type', 'Claim date', 'delta']

In [56]:
#find the order and revenue from walmart#
CF_rev_30days1 = new_df_1.groupby('order_plcd_dt').agg({'Total_Quantity' :'sum', 'Total_Revenue' : 'sum'})
CF_rev_30days1.reset_index(inplace = True)
CF_rev_30days1=CF_rev_30days1.rename(columns={'order_plcd_dt':'ORDER_PLACED_DT', 'Total_Revenue' : 'Sum(TOT_GMV)'})
CF_rev_30days1['ORDER_PLACED_DT'] = pd.to_datetime(CF_rev_30days1['ORDER_PLACED_DT'])
#CF_rev_30days1

In [57]:
#Claims_orders
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_items_orders_all_claims""")

In [58]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_items_orders_all_claims(
offer_id VARCHAR, order_placed_dt DATE, cust_id VARCHAR, order_number BIGINT, total_revenue BIGINT, total_quantity BIGINT,
placed_amount BIGINT, placed_quantity BIGINT,
total_cancel_amount BIGINT, cancelled_quantity BIGINT,
refunded_amount BIGINT, return_quantity BIGINT,
ID VARCHAR, Claim_ID VARCHAR, Created DATE, Item_ID VARCHAR, Actual_Claim_Type VARCHAR, Claim_date DATE, delta BIGINT)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

new_df_1.to_csv('item_order' +str(date.today())+'.txt',sep='|',index=False,header=False)
filepath = 'item_order' +str(date.today())+'.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_items_orders_all_claims/" + filepath)

#Claims_orders
tns_claims_items_orders = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_items_orders_all_claims""")

In [59]:
#tns_claims_items_orders

In [106]:
#selected Walmart related product#
#find the order and revenue from walmart each day#
tns_catalog_new_all = dataset.load(name="bq_connection", query="select order_plcd_dt, \
COUNT(DISTINCT order_child_nbr) as Order_Number, \
sum(gmv_order_amt) as Total_Revenue, sum(net_cncl_qty) as Total_Quantity, \
sum(gmv_plcd_amt) as Placed_Amount, sum(plcd_qty) as Placed_Quantity, \
sum(gmv_cncl_amt) as Total_cancel_Amount, sum(cncl_qty) as Cancelled_Quantity, \
sum(gmv_rfnd_amt) as Refunded_Amount, sum(rtn_rfnd_full_qty) as Return_Quantity \
from `wmt-edw-prod.WW_CREW_DL_RPT_VM.CNSLD_ORDER_ITEM` \
where order_plcd_dt <= date'"+monthend+"' and order_plcd_dt >=date'"+dt_start+"' \
and op_cmpny_cd = 'WMT.COM' and gmv_cr_ind = 1 and gmv_ind = 1 and svc_nm = 'MP' group by order_plcd_dt")

tns_catalog_new_all.columns = ['order_plcd_dt', 'Order_Number', 'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity', 'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount', 'Return_Quantity']
tns_catalog_new_all['order_plcd_dt'] = pd.to_datetime(tns_catalog_new_all['order_plcd_dt'])
tns_catalog_new_all['Total_Revenue'] = tns_catalog_new_all['Total_Revenue'].astype(float)

In [107]:
rate = pd.merge(tns_catalog_new_all, CF_rev_30days1, how = 'left', left_on = 'order_plcd_dt', right_on = 'ORDER_PLACED_DT')
rate.fillna(0, inplace = True)
cumulative_rate = rate[rate['order_plcd_dt'] >= dt_start]
cumulative_rate.fillna(0, inplace = True)
cumulative_rate = cumulative_rate.sort_values('order_plcd_dt')
cumulative_rate

,order_plcd_dt,Order_Number,Total_Revenue,Total_Quantity_x,Placed_Amount,Placed_Quantity,Total_cancel_Amount,Cancelled_Quantity,Refunded_Amount,Return_Quantity,ORDER_PLACED_DT,Total_Quantity_y,Sum(TOT_GMV)
29,2024-01-27,368974,1.723096e+07,556479,19656741.279999996,576185,958798.456119045,19706,1466986.714999993,36267,2024-01-27 00:00:00,114.0,2480.10
35,2024-01-28,393037,1.828005e+07,604687,20848682.33,624907,965048.21,20220,1603579.604999996,40225,2024-01-28 00:00:00,178.0,4501.42
76,2024-01-29,363658,1.740815e+07,564547,19892845.980000002,584518,944790.21804762,19971,1539906.859999998,38993,2024-01-29 00:00:00,196.0,4645.86
34,2024-01-30,376463,1.822693e+07,587424,20731033.310000002,607774,963813.183166668,20350,1540293.534999999,39559,2024-01-30 00:00:00,132.0,3504.23
69,2024-01-31,395749,1.931757e+07,622017,22013612.049999997,643745,1049033.943119046,21728,1647004.615000001,41371,2024-01-31 00:00:00,158.0,5239.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2024-04-17,441744,2.445377e+07,704845,25746749.150000001,727194,1120894.2,22349,172087.650000002,4603,2024-04-17 00:00:00,12.0,313.42
63,2024-04-18,436144,2.409728e+07,692492,25210295.41,712166,1020665.92,19674,92345.369999999,2361,2024-04-18 00:00:00,5.0,144.43
2,2024-04-19,419693,2.366817e+07,664426,24740289.61,683064,1039511.94,18638,32610.550000001,851,2024-04-19 00:00:00,16.0,421.09
66,2024-04-20,413370,2.279413e+07,646301,23669764.19,663469,870488.875,17168,5150.184999999,136,0,0.0,0


In [108]:
cumulative_rate['cumulative_Total_Quantity_y'] = cumulative_rate['Total_Quantity_y'].cumsum()
cumulative_rate['cumulative_Order_Number'] = cumulative_rate['Order_Number'].cumsum()
cumulative_rate['cumulative_Sum(TOT_GMV)'] = cumulative_rate['Sum(TOT_GMV)'].cumsum()
cumulative_rate['cumulative_Total_Revenue'] = cumulative_rate['Total_Revenue'].cumsum()
cumulative_rate['cumulative_order'] = cumulative_rate['cumulative_Total_Quantity_y'] / cumulative_rate['cumulative_Order_Number'] * 10000
cumulative_rate['cumulative_revenue'] =cumulative_rate['cumulative_Sum(TOT_GMV)'].astype(float) / cumulative_rate[('cumulative_Total_Revenue')] * 10000 


In [109]:
cumulative_rate['cumulative_revenue'] =cumulative_rate['cumulative_Sum(TOT_GMV)'].astype(float) / cumulative_rate[('cumulative_Total_Revenue')] * 10000 


In [110]:
#All Claims#
cumulative_rate_sheet = cumulative_rate[['order_plcd_dt', 'Total_Quantity_y', 'Sum(TOT_GMV)', 'Order_Number', 'Total_Revenue', 'cumulative_order', 'cumulative_revenue']]
cumulative_rate_sheet.rename(columns={'order_placed_dt':'day',
                   'Total_Quantity_y':'Counterfeits Orders',
                   'Sum(TOT_GMV)':'Counterfeits Revenue','Order_Number':'Catalog Totals Orders','Total_Revenue':'Catalog Totals Revenue',
                   'cumulative_order':'Order Basis Points(YTD)',
                   'cumulative_revenue':'Revenue Basis Points(YTD)'}, inplace = True)
cumulative_rate_sheet

/libs/project/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,order_plcd_dt,Counterfeits Orders,Counterfeits Revenue,Catalog Totals Orders,Catalog Totals Revenue,Order Basis Points(YTD),Revenue Basis Points(YTD)
29,2024-01-27,114.0,2480.10,368974,1.723096e+07,3.089649,1.439328
35,2024-01-28,178.0,4501.42,393037,1.828005e+07,3.831966,1.966016
76,2024-01-29,196.0,4645.86,363658,1.740815e+07,4.335200,2.197197
34,2024-01-30,132.0,3504.23,376463,1.822693e+07,4.127467,2.126837
69,2024-01-31,158.0,5239.37,395749,1.931757e+07,4.099309,2.251841
...,...,...,...,...,...,...,...
38,2024-04-17,12.0,313.42,441744,2.445377e+07,1.835768,1.031663
63,2024-04-18,5.0,144.43,436144,2.409728e+07,1.814115,1.018248
2,2024-04-19,16.0,421.09,419693,2.366817e+07,1.796976,1.007005
66,2024-04-20,0.0,0,413370,2.279413e+07,1.776051,0.994196


In [111]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_catalog_table""")

In [112]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_catalog_table(
order_placed_dt DATE, order_number BIGINT, total_revenue BIGINT, Catalog_totals_orders BIGINT, Catalog_Total_Revenue DOUBLE,
Order_Basis_Points_YTD DOUBLE, Revenue_Basis_Points_YTD DOUBLE, label VARCHAR)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

In [113]:
tns_catalog_new = cumulative_rate_sheet

tns_catalog_new ["label"]="CF Rate(All Claims)"

tns_catalog_new.to_csv('catalog_order' +str(date.today())+'.txt',sep='|',index=False,header=False)
filepath = 'catalog_order' +str(date.today())+'.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_catalog_table/" + filepath)
all_claims = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_catalog_table""")

/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [114]:
###All claims###
#all_claims

## Why is this hardcoded??

### Rights Owner(Customer Report)

In [115]:
#excluded-claims_set#
excluded_claims_set = np.array(['IPRR0035665',
 'IPRR0035666',
 'IPRR0035667',
 'IPRR0035786',
 'IPRR0035990',
 'IPRR0036481',
 'IPRR0036522',
 'IPRR0036825',
 'IPRR0036826',
 'IPRR0036834',
 'IPRR0036836',
 'IPRR0037009',
 'IPRR0037023',
 'IPRR0037048',
 'IPRR0037080',
 'IPRR0037420',
 'IPRR0037442',
 'IPRR0037443',
 'IPRR0037445',
 'IPRR0037449',
 'IPRR0037583',
 'IPRR0038000',
 'IPRR0038017',
 'IPRR0038018',
 'IPRR0038023',
 'IPRR0038575',
 'IPRR0038581',
 'IPRR0038583',
 'IPRR0038584',
 'IPRR0038585',
 'IPRR0038587',
 'IPRR0038590',
 'IPRR0038857',
 'IPRR0038976',
 'IPRR0039192',
 'IPRR0039195',
 'IPRR0039196',
 'IPRR0039533',
 'IPRR0039534',
 'IPRR0039803',
 'IPRR0039806',
 'IPRR0039823',
 'IPRR0040217',
 'IPRR0040408',
 'IPRR0040412',
 'IPRR0040418',
 'IPRR0040428',
 'IPRR0040717',
 'IPRR0040885',
 'IPRR0040900',
 'IPRR0041233',
 'IPRR0041236',
 'IPRR0041246',
 'IPRR0041248',
 'IPRR0041249',
 'IPRR0041251',
 'IPRR0041366',
 'IPRR0040982',
 'IPRR0040574'])

In [116]:
##claim_to_remove-Customer Report##
claim_to_remove_table = claim_table[claim_table['Claim ID'].isin(excluded_claims_set)]["Claim ID"]
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_remove_items_orders""")

In [117]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_remove_items_orders(
Claim_ID VARCHAR )
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)
claim_to_remove_table.to_csv('item_order' +str(date.today())+'_claimremoved.txt',sep='|',index=False,header=False)
filepath = 'item_order' +str(date.today())+'_claimremoved.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_remove_items_orders/" + filepath)
tns_claims_remove = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_remove_items_orders""")
tns_claims_remove.shape

(0, 1)

In [118]:
##claim table without customer report##
claim_table2 = claim_table1[~claim_table1['Claim ID'].isin(excluded_claims_set)]
claim_table2 = claim_table2[(claim_table2['Reported Details'].str.contains('CUSTOMER REPORT')==False)&(claim_table2['IP Company Name'].str.contains('CUSTOMER REPORT')==False)]
claim_table2

,Claim ID,Reported Claim Type,Actual Claim Type,Brand,IP Company Name,Email Address,Reported Sellers,Item ID,Claim State,Action,Created,Updated,Legacy Seller ID,Reason Code,Category,Offer Publish Status,Decision Type,ID,Reported Details,Country
81,IPRR0243398,Counterfeit,Confirmed Counterfeit,,,walmartmaster@truskin.com,NB Daily Goods Supplies,878778137,Closed,Block,2024-02-01,2024-02-02 06:19:20+00:00,101573410,IP-Counterfeit High-Severity,Health &amp; Beauty,Published,Seller Level,8A8CD8245F3330DF971EA2E21E4F0412,"The item received in order 200011583170340, fr...",
100,IPRR0246200,Counterfeit,Confirmed Counterfeit,PRESERVISION,BAUSCH & LOMB INCORPORATED,kristina.schrader@arlaw.com,Southsea Health Shop,5244995391,Closed,Block,2024-02-23,2024-02-23 22:43:41+00:00,101494956,IP-Counterfeit High-Severity,Health &amp; Beauty,Published,Seller Level,70EBED11BCDD3D2CABC93859B546B709,This firm represents Bausch & Lomb Incorporate...,USA
138,IPRR0249293,Counterfeit,Confirmed Counterfeit,,,awilkins@ppibeauty.com,Oktake,229747348,Closed,Block,2024-03-13,2024-03-14 00:43:06+00:00,101020688,IP-Counterfeit High-Severity,Health &amp; Beauty,Published,Seller Level,8F006188F6973261AEBF570C588F1AB9,"This product is not authentic, it is missing v...",
146,IPRR0257542,Counterfeit,Counterfeit,Apple,Apple Inc,mrkt-apple@tracer.ai,"DreamTumn shop,DreamTumn shop,DreamTumn shop,D...",5587767958,Closed,Block,2024-04-12,2024-04-17 15:52:57+00:00,101525241,IP-Design Rights High-Severity,Electronics,Published,Seller Level,35E50295771A31F493F7AE0C28297390,Item infringes US - TM - AirPod Trade Dress - ...,USA
183,IPRR0259439,Counterfeit,Confirmed Counterfeit,lululemon,"lululemon athletica, inc.",ipenforcementgroup12@sideman.com,Ayla goods,3235157718,Closed,Block,2024-04-19,2024-04-19 23:35:02+00:00,101397477,IP-Counterfeit High-Severity,"Clothing, Shoes &amp; Accessories",Published,Seller Level,AEDA23CE33F339CDA0B7AECE09B818E6,We are the authorized representative of lulule...,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6701,IPRR0259440,Counterfeit,Confirmed Counterfeit,lululemon,"lululemon athletica, inc.",ipenforcementgroup12@sideman.com,BearVe,3235157718,Closed,Block,2024-04-19,2024-04-20 00:30:10+00:00,101554514,IP-Counterfeit High-Severity,"Clothing, Shoes &amp; Accessories",Published,Seller Level,66EB9FE4295831B68A5D009A87582554,We are the authorized representative of lulule...,United States
6746,IPRR0250383,Counterfeit,Counterfeit,lululemon,Internal escalation,william.zuniga@walmart.com,"Meem Traders,Do Good Partners,EZ Trading LLC",3235157718,Closed,Block,2024-03-18,2024-03-18 21:51:03+00:00,101117758,IP-Counterfeit Low-Severity,"Clothing, Shoes &amp; Accessories",Published,Seller Level,8621395404A9388EB032783F3D40AA81,"Internal escalation, please work as suspected ...",USA
6749,IPRR0250383,Counterfeit,Counterfeit,lululemon,Internal escalation,william.zuniga@walmart.com,"Meem Traders,Do Good Partners,EZ Trading LLC",3235157718,Closed,Block,2024-03-18,2024-03-18 21:51:03+00:00,101544731,IP-Counterfeit Low-Severity,"Clothing, Shoes &amp; Accessories",Published,Seller Level,66C88462B09F3818AEED1DD856B5BD24,"Internal escalation, please work as suspected ...",USA
6766,IPRR0243514,Counterfeit,Confirmed Counterfeit,Jones & Bartlett Learning,Jones & Bartlett Learning,noi@bcguardian.com,Unique-store,1516679416,Closed,Block,2024-02-02,2024-02-03 07:21:59+00:00,101486847,IP-Counterfeit High-Severity,"Books, Music &amp; Movies",Published,Seller Level,B689770DAE473892B1D1F14AC2B22B44,This item is counterfeit. We have completed a ...,USA


In [119]:
tns_claim_order2=pd.merge(tns_claim_order, claim_table2.drop_duplicates(subset = 'ID'), left_on = 'offer_id', right_on = 'ID')
tns_claim_order2=tns_claim_order2[['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number',
       'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity',
       'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount','Return_Quantity']]
tns_claim_order2

,offer_id,order_plcd_dt,cust_id,Order_Number,Total_Revenue,Total_Quantity,Placed_Amount,Placed_Quantity,Total_cancel_Amount,Cancelled_Quantity,Refunded_Amount,Return_Quantity
0,36F8C68C50B93B0B91B6CC11B7AA4980,2024-04-01,8BE238773EDA4C119A7C4218961BA40F,1,98.7,1,98.7,1,0,0.0,0,0
1,36F8C68C50B93B0B91B6CC11B7AA4980,2024-04-14,48D23D8EEA8E41F89BC9B4AD73A7DFF6,1,189.88,2,189.88,2,0,0.0,0,0
2,36F8C68C50B93B0B91B6CC11B7AA4980,2024-04-03,340C166BC3A54817B4ECA52EA40AE1CC,1,193.9,2,193.9,2,0,0.0,0,0
3,36F8C68C50B93B0B91B6CC11B7AA4980,2024-04-12,A820E867030C4679824277AA1E8EE477,1,189.88,2,189.88,2,0,0.0,0,0
4,36F8C68C50B93B0B91B6CC11B7AA4980,2024-04-03,1848C5E0A9E84F0B98861BCBCD22D472,1,193.9,2,193.9,2,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4706,930E1C03B43131BB9454ABECA31C1F6F,2024-01-31,B267B5BFD7824918AC384CD46817A6B3,1,45.99,1,45.99,1,0,0.0,0,0
4707,930E1C03B43131BB9454ABECA31C1F6F,2024-02-07,A4D6DB16380F48C98D199E1D76E9E69B,1,45.99,1,45.99,1,0,0.0,0,0
4708,43FD6177AE60433B872E82B268904D91,2024-03-23,E49AD1DD6A9F472A8663D3FB08586F57,1,29.95,1,29.95,1,0,0.0,0,0
4709,43FD6177AE60433B872E82B268904D91,2024-02-23,F0738EF89B566718E04359D20F0A9E45,1,29.95,1,29.95,1,0,0.0,0,0


In [120]:
#tns_claim_order2.info()

In [121]:
Claim_df2 = claim_table2.groupby('ID').agg({'Claim ID' : 'first', 'Created' :'first', 'Item ID' :'first', 'Actual Claim Type' : 'first'})
Claim_df2 = Claim_df2.reset_index()
new_df2 = pd.merge(tns_claim_order2, Claim_df2.drop_duplicates(subset = 'ID'), left_on = 'offer_id', right_on = 'ID', how = 'left')
new_df2['Claim date']= pd.to_datetime(new_df2['Created']).dt.date
new_df2['order_plcd_dt']= pd.to_datetime(new_df2['order_plcd_dt']).dt.date
new_df2['delta'] = (new_df2['Claim date'] - new_df2['order_plcd_dt']).dt.days
new_df_2 = new_df2[(new_df2['delta'] >= 0) & (new_df2['delta'] <= 30)]
new_df_2.columns = ['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number', 'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity', 'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount', 'Return_Quantity', 'ID', 'Item ID', 'Claim ID', 'Created', 'Actual Claim Type', 'Claim date', 'delta']

CF_rev_30days2 = new_df_2.groupby('order_plcd_dt').agg({'Total_Quantity' :'sum', 'Total_Revenue' : 'sum'})
CF_rev_30days2.reset_index(inplace = True)
CF_rev_30days2=CF_rev_30days2.rename(columns={'order_plcd_dt':'ORDER_PLACED_DT', 'Total_Revenue' : 'Sum(TOT_GMV)'})
CF_rev_30days2['ORDER_PLACED_DT'] = pd.to_datetime(CF_rev_30days2['ORDER_PLACED_DT'])



In [122]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_items_orders_rights_owner""")
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_items_orders_rights_owner(
offer_id VARCHAR, order_placed_dt DATE, cust_id VARCHAR, order_number BIGINT, total_revenue BIGINT, total_quantity BIGINT,
placed_amount BIGINT, placed_quantity BIGINT,
total_cancel_amount BIGINT, cancelled_quantity BIGINT,
refunded_amount BIGINT, return_quantity BIGINT,
ID VARCHAR, Claim_ID VARCHAR, Created DATE, Item_ID VARCHAR, Actual_Claim_Type VARCHAR, Claim_date DATE, delta BIGINT)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

new_df_2.to_csv('item_order' +str(date.today())+'_rights_owner.txt',sep='|',index=False,header=False)
filepath = 'item_order' +str(date.today())+'_rights_owner.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_items_orders_rights_owner/" + filepath)

tns_claims_items_orders2 = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_items_orders_rights_owner""")
tns_claims_items_orders2.shape

(3754, 19)

In [123]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_catalog_table_no_customer""")

In [124]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_catalog_table_no_customer(
order_placed_dt DATE, order_number BIGINT, total_revenue BIGINT, Catalog_totals_orders BIGINT, Catalog_Total_Revenue DOUBLE,
Order_Basis_Points_YTD DOUBLE, Revenue_Basis_Points_YTD DOUBLE, label VARCHAR)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

###import tns_catalog_new table,then combine two tables together#
rate2 = pd.merge(tns_catalog_new_all, CF_rev_30days2, how = 'left', left_on = 'order_plcd_dt', right_on = 'ORDER_PLACED_DT')
rate2.fillna(0, inplace = True)
cumulative_rate2 = rate2[rate2['order_plcd_dt'] >= dt_start]
cumulative_rate2.fillna(0, inplace = True)
cumulative_rate2 = cumulative_rate2.sort_values('order_plcd_dt')
cumulative_rate2['cumulative_Total_Quantity_y'] = cumulative_rate2['Total_Quantity_y'].cumsum()
cumulative_rate2['cumulative_Order_Number'] = cumulative_rate2['Order_Number'].cumsum()
cumulative_rate2['cumulative_Sum(TOT_GMV)'] = cumulative_rate2['Sum(TOT_GMV)'].cumsum()
cumulative_rate2['cumulative_Total_Revenue'] = cumulative_rate2['Total_Revenue'].cumsum()


In [125]:
cumulative_rate2['cumulative_order'] = cumulative_rate2['cumulative_Total_Quantity_y'] / cumulative_rate2['cumulative_Order_Number'] * 10000
cumulative_rate2['cumulative_revenue'] = cumulative_rate2['cumulative_Sum(TOT_GMV)'].astype(float) / cumulative_rate2['cumulative_Total_Revenue'] * 10000

cumulative_rate_sheet2 = cumulative_rate2[['order_plcd_dt', 'Total_Quantity_y', 'Sum(TOT_GMV)', 'Order_Number', 'Total_Revenue', 'cumulative_order', 'cumulative_revenue']]
cumulative_rate_sheet2.rename(columns={'order_plcd_dt':'day',
                   'Total_Quantity_y':'Counterfeits Orders',
                   'Sum(TOT_GMV)':'Counterfeits Revenue','Order_Number':'Catalog Totals Orders','Total_Revenue':'Catalog Totals Revenue',
                   'cumulative_order':'Order Basis Points(YTD)',
                   'cumulative_revenue':'Revenue Basis Points(YTD)'}, inplace = True)

tns_catalog_new2 = cumulative_rate_sheet2

tns_catalog_new2["label"]="CF Rate(Rights Owner)"
tns_catalog_new2.to_csv('catalog_order' +str(date.today())+'_NoCustomer.txt',sep='|',index=False,header=False)

/libs/project/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [126]:
filepath = 'catalog_order' +str(date.today())+'_NoCustomer.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_catalog_table_no_customer/" + filepath)
rights_owner = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_catalog_table_no_customer""")
rights_owner

,order_placed_dt,order_number,total_revenue,catalog_totals_orders,catalog_total_revenue,order_basis_points_ytd,revenue_basis_points_ytd,label
0,2024-01-27,79,1747,368974,1.723096e+07,2.141072,1.014447,CF Rate(Rights Owner)
1,2024-01-28,111,2299,393037,1.828005e+07,2.493402,1.139863,CF Rate(Rights Owner)
2,2024-01-29,159,3066,363658,1.740815e+07,3.100379,1.344279,CF Rate(Rights Owner)
3,2024-01-30,98,2193,376463,1.822693e+07,2.975770,1.308234,CF Rate(Rights Owner)
4,2024-01-31,88,2064,395749,1.931757e+07,2.818933,1.257137,CF Rate(Rights Owner)
...,...,...,...,...,...,...,...,...
81,2024-04-17,12,313,441744,2.445377e+07,1.255007,0.688622,CF Rate(Rights Owner)
82,2024-04-18,5,144,436144,2.409728e+07,1.240660,0.679942,CF Rate(Rights Owner)
83,2024-04-19,16,421,419693,2.366817e+07,1.230380,0.673226,CF Rate(Rights Owner)
84,2024-04-20,0,0,413370,2.279413e+07,1.216053,0.664662,CF Rate(Rights Owner)


### Confirmed Counterfeit (High Risk)#confirmed#

In [127]:
claim_table3 = claim_table1[~claim_table1['Claim ID'].isin(excluded_claims_set)]
claim_table3 = claim_table3[claim_table3['Actual Claim Type'] == 'Confirmed Counterfeit']

In [128]:
tns_claim_order3=pd.merge(tns_claim_order, claim_table3.drop_duplicates(subset = 'ID'), left_on = 'offer_id', right_on = 'ID')
tns_claim_order3=tns_claim_order3[['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number',
       'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity',
       'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount',
       'Return_Quantity']]

In [129]:
Claim_df3 = claim_table3.groupby('ID').agg({'Claim ID' : 'first', 'Created' :'first', 'Item ID' :'first', 'Actual Claim Type' : 'first'})
Claim_df3 = Claim_df3.reset_index()

new_df3 = pd.merge(tns_claim_order3, Claim_df3, left_on = 'offer_id', right_on = 'ID', how = 'left')
new_df3['Claim date']= pd.to_datetime(new_df3['Created']).dt.date
new_df3['order_plcd_dt']= pd.to_datetime(new_df3['order_plcd_dt']).dt.date
new_df3['delta'] = (new_df3['Claim date'] - new_df3['order_plcd_dt']).dt.days
new_df_3 = new_df3[(new_df3['delta'] >= 0) & (new_df3['delta'] <= 30)]
new_df_3.columns = ['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number', 'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity', 'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount', 'Return_Quantity', 'ID', 'Item ID', 'Claim ID', 'Created', 'Actual Claim Type', 'Claim date', 'delta']

CF_rev_30days3 = new_df_3.groupby('order_plcd_dt').agg({'Total_Quantity' :'sum', 'Total_Revenue' : 'sum'})
CF_rev_30days3.reset_index(inplace = True)
CF_rev_30days3=CF_rev_30days3.rename(columns={'order_plcd_dt':'ORDER_PLACED_DT', 'Total_Revenue' : 'Sum(TOT_GMV)'})
CF_rev_30days3['ORDER_PLACED_DT'] = pd.to_datetime(CF_rev_30days3['ORDER_PLACED_DT'])

connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_items_orders_high_risk""")
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_items_orders_high_risk(
offer_id VARCHAR, order_placed_dt DATE, cust_id VARCHAR, order_number BIGINT, total_revenue BIGINT, total_quantity BIGINT,
placed_amount BIGINT, placed_quantity BIGINT,
total_cancel_amount BIGINT, cancelled_quantity BIGINT,
refunded_amount BIGINT, return_quantity BIGINT,
ID VARCHAR, Claim_ID VARCHAR, Created DATE, Item_ID VARCHAR, Actual_Claim_Type VARCHAR, Claim_date DATE, delta BIGINT)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

new_df_3.to_csv('item_order' +str(date.today())+'_high_risk.txt',sep='|',index=False,header=False)
filepath = 'item_order' +str(date.today())+'_high_risk.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_items_orders_high_risk/" + filepath)
tns_claims_items_orders3 = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_items_orders_high_risk""")

connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_catalog_table_Confirmed_only""")
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_catalog_table_Confirmed_only(
order_placed_dt DATE, order_number BIGINT, total_revenue BIGINT, Catalog_totals_orders BIGINT, Catalog_Total_Revenue DOUBLE,
Order_Basis_Points_YTD DOUBLE, Revenue_Basis_Points_YTD DOUBLE, label VARCHAR)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

###import tns_catalog_new table,then combine two tables together#
rate3 = pd.merge(tns_catalog_new_all, CF_rev_30days3, how = 'left', left_on = 'order_plcd_dt', right_on = 'ORDER_PLACED_DT')
rate3.fillna(0, inplace = True)
cumulative_rate3 = rate3[rate3['order_plcd_dt'] >= dt_start]
cumulative_rate3.fillna(0, inplace = True)
cumulative_rate3 = cumulative_rate3.sort_values('order_plcd_dt')
cumulative_rate3['cumulative_Total_Quantity_y'] = cumulative_rate3['Total_Quantity_y'].cumsum()
cumulative_rate3['cumulative_Order_Number'] = cumulative_rate3['Order_Number'].cumsum()
cumulative_rate3['cumulative_Sum(TOT_GMV)'] = cumulative_rate3['Sum(TOT_GMV)'].cumsum()
cumulative_rate3['cumulative_Total_Revenue'] = cumulative_rate3['Total_Revenue'].cumsum()
cumulative_rate3['cumulative_order'] = cumulative_rate3['cumulative_Total_Quantity_y'] / cumulative_rate3['cumulative_Order_Number'] * 10000
cumulative_rate3['cumulative_revenue'] = cumulative_rate3['cumulative_Sum(TOT_GMV)'].astype(float) / cumulative_rate3['cumulative_Total_Revenue'] * 10000

cumulative_rate_sheet3 = cumulative_rate3[['order_plcd_dt', 'Total_Quantity_y', 'Sum(TOT_GMV)', 'Order_Number', 'Total_Revenue', 'cumulative_order', 'cumulative_revenue']]
cumulative_rate_sheet3.rename(columns={'order_plcd_dt':'day',
                   'Total_Quantity_y':'Counterfeits Orders',
                   'Sum(TOT_GMV)':'Counterfeits Revenue','Order_Number':'Catalog Totals Orders','Total_Revenue':'Catalog Totals Revenue',
                   'cumulative_order':'Order Basis Points(YTD)',
                   'cumulative_revenue':'Revenue Basis Points(YTD)'}, inplace = True)

tns_catalog_new3 = cumulative_rate_sheet3

tns_catalog_new3["label"]="CF Rate(High Risk)"
tns_catalog_new3.to_csv('catalog_order' +str(date.today())+'_confirmed_only.txt',sep='|',index=False,header=False)
filepath = 'catalog_order' +str(date.today())+'_confirmed_only.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_catalog_table_confirmed_only/" + filepath)

/libs/project/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [130]:
high_risk = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_catalog_table_Confirmed_only""")
high_risk

,order_placed_dt,order_number,total_revenue,catalog_totals_orders,catalog_total_revenue,order_basis_points_ytd,revenue_basis_points_ytd,label
0,2024-01-27,51,1212,368974,1.723096e+07,1.382211,0.703704,CF Rate(High Risk)
1,2024-01-28,85,1803,393037,1.828005e+07,1.784751,0.849342,CF Rate(High Risk)
2,2024-01-29,86,1715,363658,1.740815e+07,1.972161,0.894052,CF Rate(High Risk)
3,2024-01-30,77,1779,376463,1.822693e+07,1.990504,0.915145,CF Rate(High Risk)
4,2024-01-31,86,2040,395749,1.931757e+07,2.028578,0.945265,CF Rate(High Risk)
...,...,...,...,...,...,...,...,...
81,2024-04-17,12,313,441744,2.445377e+07,1.070962,0.613838,CF Rate(High Risk)
82,2024-04-18,5,144,436144,2.409728e+07,1.058931,0.606191,CF Rate(High Risk)
83,2024-04-19,16,421,419693,2.366817e+07,1.050825,0.600461,CF Rate(High Risk)
84,2024-04-20,0,0,413370,2.279413e+07,1.038589,0.592823,CF Rate(High Risk)


### Customer report

In [131]:
##claim_to_remove-Customer Report##
claim_to_keep_table = claim_table[~claim_table['Claim ID'].isin(excluded_claims_set)]["Claim ID"]
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_keep_items_orders""")

In [132]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_keep_items_orders(
Claim_ID VARCHAR )
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)
claim_to_keep_table.to_csv('item_order' +str(date.today())+'_claimkeeped.txt',sep='|',index=False,header=False)
filepath = 'item_order' +str(date.today())+'_claimkeeped.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_keep_items_orders/" + filepath)
tns_claims_keep = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_keep_items_orders""")
tns_claims_keep.shape

(6912, 1)

In [133]:
##claim table without customer report##
claim_table4 = claim_table1[claim_table1['Claim ID'].isin(excluded_claims_set)|(claim_table1['Reported Details'].str.contains('CUSTOMER REPORT')==True)|(claim_table1['IP Company Name'].str.contains('CUSTOMER REPORT')==True)]
#claim_table4 =  claim_table1[claim_table1['Claim ID'].isin(excluded_claims_set)]|claim_table4[(claim_table4['Reported Details'].str.contains('CUSTOMER REPORT')==True)|(claim_table2['IP Company Name'].str.contains('CUSTOMER REPORT')==True)]
claim_table4

,Claim ID,Reported Claim Type,Actual Claim Type,Brand,IP Company Name,Email Address,Reported Sellers,Item ID,Claim State,Action,Created,Updated,Legacy Seller ID,Reason Code,Category,Offer Publish Status,Decision Type,ID,Reported Details,Country
305,IPRR0242952,Counterfeit,Counterfeit,Paco Rabanne,CUSTOMER REPORT – BAZAARVOICE,sharon.baxter@walmart.com,FragranceShop.com,28544773,Closed,Block,2024-01-31,2024-01-31 17:18:42+00:00,101012476,IP-Counterfeit Low-Severity,Health &amp; Beauty,Published,Seller Level,584E29DA60194B9E8F18FB2C61B41876,"""Please work as SCF -- 3rd party fulfillment o...",United States
654,IPRR0243856,Counterfeit,Counterfeit,Laneige,CUSTOMER REPORT - BVR,ashley.veloz@walmart.com,Top Deals,518469376,Closed,Block,2024-02-06,2024-02-06 23:28:00+00:00,15841,IP-Counterfeit Low-Severity,Health &amp; Beauty,Published,Seller Level,FEC70AC726E94ADE98F461533D1A3EC2,Please work as SCF -- 3rd party fulfillment or...,United States
661,IPRR0252822,Counterfeit,Counterfeit,Sol De Janeiro,CUSTOMER REPORT – BAZAARVOICE,sharon.baxter@walmart.com,Jay's Skincare and Wellness,2992285176,Closed,Block,2024-03-26,2024-03-26 19:35:42+00:00,101573967,IP-Counterfeit Low-Severity,Health &amp; Beauty,Published,Seller Level,5CAA01F751CB3BEB9230A7B9C1FD5C47,"""Please work as SCF -- 3rd party fulfillment o...",United States
808,IPRR0244765,Counterfeit,Counterfeit,Stanley,CUSTOMER REPORT – BAZAARVOICE,sharon.baxter@walmart.com,"859flips,Mayubeautify,Plunder from Down Under,...",5104206246,Closed,Block,2024-02-14,2024-02-14 17:54:35+00:00,101228488,IP-Counterfeit Low-Severity,Home &amp; Garden,Published,Seller Level,A7DF7412324432DEA41DE7FF924E068F,"""Please work as SCF -- 3rd party fulfillment o...",United States
884,IPRR0247745,Counterfeit,Counterfeit,Gerber,CUSTOMER REPORT – BAZAARVOICE,sharon.baxter@walmart.com,Vincero Trading LLC,5278058983,Closed,Block,2024-03-05,2024-03-20 23:02:35+00:00,101465859,IP-Counterfeit Low-Severity,Sports &amp; Outdoors,Published,Seller Level,CA247B88CD1B37529A051825F7AC4542,"""Please work as SCF -- 3rd party fulfillment o...",United States
895,IPRR0244765,Counterfeit,Counterfeit,Stanley,CUSTOMER REPORT – BAZAARVOICE,sharon.baxter@walmart.com,"859flips,Mayubeautify,Plunder from Down Under,...",2242956312,Closed,Block,2024-02-14,2024-02-14 17:52:22+00:00,101405769,IP-Counterfeit Low-Severity,Home &amp; Garden,Published,Seller Level,AAD77486737E34B4A38A15028112B7E8,"""Please work as SCF -- 3rd party fulfillment o...",United States
974,IPRR0243967,Counterfeit,Counterfeit,Britney Spears,CUSTOMER REPORT - BVR,ashley.veloz@walmart.com,Perfume Empire,779590629,Closed,Block,2024-02-07,2024-02-07 16:20:05+00:00,101074054,IP-Counterfeit Low-Severity,Health &amp; Beauty,Published,Seller Level,4B0B5D40614E35C8ABF37534781EC11C,Please work as SCF -- 3rd party fulfillment or...,United States
1505,IPRR0243300,Counterfeit,Counterfeit,Yves Saint Laurent,CUSTOMER REPORT - BVR,ashley.veloz@walmart.com,"USA Wholesale Center,OC Fashion Jewelry Inc.",5298521578,Closed,Block,2024-02-01,2024-02-01 19:49:45+00:00,101394426,IP-Counterfeit Low-Severity,Health &amp; Beauty,Published,Seller Level,63304E6818E635B3B9224A1EB9ECE170,Please work as SCF -- 3rd party fulfillment or...,United States
1530,IPRR0243094,Counterfeit,Counterfeit,Estee Lauder,CUSTOMER REPORT - BVR,ashley.veloz@walmart.com,"S.N. Watches INC.,KJ WHOLESALE LLC",853264431,Closed,Block,2024-01-31,2024-02-01 02:11:36+00:00,101210466,IP-Counterfeit Low-Severity,Health &amp; Beauty,Published,Seller Level,B94A14529A2637D4A31260AC9AF862E2,Please work as SCF -- 3rd party fulfillment or...,United States
1642,IPRR0246787,Counterfeit,Counterfeit,Versace,CUSTOMER REPORT – BAZAARVOICE,sharon.baxter@walmart.com,2H Commerce,34041822,Closed,Block,2024-02-28,2024-02-28 17:55:14+00:00,101484424,IP-Counterfeit Low-Severity,Health &amp; Beauty,Published,Seller Level,29072EF41B043981910D3B88617312E4,"""Please work as SCF -- 3rd party fulfillment o...",United States


In [134]:
tns_claim_order4=pd.merge(tns_claim_order, claim_table4.drop_duplicates(subset = 'ID'), left_on = 'offer_id', right_on = 'ID')
tns_claim_order4=tns_claim_order4[['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number',
       'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity',
       'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount','Return_Quantity']]
tns_claim_order4

,offer_id,order_plcd_dt,cust_id,Order_Number,Total_Revenue,Total_Quantity,Placed_Amount,Placed_Quantity,Total_cancel_Amount,Cancelled_Quantity,Refunded_Amount,Return_Quantity
0,81A6DC7DD1753355A51217D7C3CD787E,2024-03-11,FB02BB801BD64DE28C441C3809F6A57A,1,13.96,2,13.96,2,0,0.0,0,0
1,81A6DC7DD1753355A51217D7C3CD787E,2024-02-29,11C515755D1446539F8110541C509E7B,1,7.9,1,7.9,1,0,0.0,0,0
2,81A6DC7DD1753355A51217D7C3CD787E,2024-02-29,D4837E6EF57749288C1356528F423772,1,7.9,1,7.9,1,0,0.0,0,0
3,81A6DC7DD1753355A51217D7C3CD787E,2024-02-24,8870DDAA8EFF4883A6346627C2818ED3,1,7.99,1,7.99,1,0,0.0,0,0
4,81A6DC7DD1753355A51217D7C3CD787E,2024-02-26,D57B3EE5E7AF4363B73AB13F175912A8,1,8.79,1,8.79,1,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2965,1061567144493FC08045BEB44A9FACB7,2024-01-28,66E7A9492F8C4BD3B6C6B9ECF2CD8145,1,39.99,1,39.99,1,0,0.0,0,0
2966,1061567144493FC08045BEB44A9FACB7,2024-01-29,6A3AAC91848E40E3B8821347C0919468,1,39.99,1,39.99,1,0,0.0,0,0
2967,1061567144493FC08045BEB44A9FACB7,2024-01-28,F05F22E021DD04A1E044001517F43A86,1,39.99,1,39.99,1,0,0.0,0,0
2968,1061567144493FC08045BEB44A9FACB7,2024-01-28,F026371BC2732B39E044001517F43A86,1,39.99,1,39.99,1,0,0.0,0,0


In [135]:
Claim_df4 = claim_table4.groupby('ID').agg({'Claim ID' : 'first', 'Created' :'first', 'Item ID' :'first', 'Actual Claim Type' : 'first'})
Claim_df4 = Claim_df4.reset_index()
new_df4 = pd.merge(tns_claim_order4, Claim_df4, left_on = 'offer_id', right_on = 'ID', how = 'left')
new_df4['Claim date']= pd.to_datetime(new_df4['Created']).dt.date
new_df4['order_plcd_dt']= pd.to_datetime(new_df4['order_plcd_dt']).dt.date
new_df4['delta'] = (new_df4['Claim date'] - new_df4['order_plcd_dt']).dt.days
new_df_4 = new_df4[(new_df4['delta'] >= 0) & (new_df4['delta'] <= 30)]
new_df_4.columns = ['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number', 'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity', 'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount', 'Return_Quantity', 'ID', 'Item ID', 'Claim ID', 'Created', 'Actual Claim Type', 'Claim date', 'delta']

CF_rev_30days4 = new_df_4.groupby('order_plcd_dt').agg({'Total_Quantity' :'sum', 'Total_Revenue' : 'sum'})
CF_rev_30days4.reset_index(inplace = True)
CF_rev_30days4=CF_rev_30days4.rename(columns={'order_plcd_dt':'ORDER_PLACED_DT', 'Total_Revenue' : 'Sum(TOT_GMV)'})
CF_rev_30days4['ORDER_PLACED_DT'] = pd.to_datetime(CF_rev_30days2['ORDER_PLACED_DT'])



In [136]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_items_orders_customer_report""")

In [137]:

connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_items_orders_customer_report(
offer_id VARCHAR, order_placed_dt DATE, cust_id VARCHAR, order_number BIGINT, total_revenue BIGINT, total_quantity BIGINT,
placed_amount BIGINT, placed_quantity BIGINT,
total_cancel_amount BIGINT, cancelled_quantity BIGINT,
refunded_amount BIGINT, return_quantity BIGINT,
ID VARCHAR, Claim_ID VARCHAR, Created DATE, Item_ID VARCHAR, Actual_Claim_Type VARCHAR, Claim_date DATE, delta BIGINT)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

new_df_4.to_csv('item_order' +str(date.today())+'_customer_report.txt',sep='|',index=False,header=False)
filepath = 'item_order' +str(date.today())+'_customer_report.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_items_orders_customer_report/" + filepath)

tns_claims_items_orders4 = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_items_orders_customer_report""")
tns_claims_items_orders4.shape

(2109, 19)

In [138]:
tns_claims_items_orders4.head()

,offer_id,order_placed_dt,cust_id,order_number,total_revenue,total_quantity,placed_amount,placed_quantity,total_cancel_amount,cancelled_quantity,refunded_amount,return_quantity,id,claim_id,created,item_id,actual_claim_type,claim_date,delta
0,81A6DC7DD1753355A51217D7C3CD787E,2024-03-11,FB02BB801BD64DE28C441C3809F6A57A,1,13,2,13,2,0,0,0,0,81A6DC7DD1753355A51217D7C3CD787E,IPRR0256974,2024-04-10,276080633,Counterfeit,2024-04-10,30
1,81A6DC7DD1753355A51217D7C3CD787E,2024-03-30,F18B5A18C0116F3EE044001517F43A86,1,9,1,9,1,0,0,0,0,81A6DC7DD1753355A51217D7C3CD787E,IPRR0256974,2024-04-10,276080633,Counterfeit,2024-04-10,11
2,81A6DC7DD1753355A51217D7C3CD787E,2024-04-10,04E8E5C3243E4CC68D44FB4FAB8E4DB3,1,9,1,9,1,0,0,0,0,81A6DC7DD1753355A51217D7C3CD787E,IPRR0256974,2024-04-10,276080633,Counterfeit,2024-04-10,0
3,81A6DC7DD1753355A51217D7C3CD787E,2024-04-08,5BA810EA3D614DBE9B1C32F7FDDE5A6C,1,9,1,9,1,0,0,0,0,81A6DC7DD1753355A51217D7C3CD787E,IPRR0256974,2024-04-10,276080633,Counterfeit,2024-04-10,2
4,81A6DC7DD1753355A51217D7C3CD787E,2024-04-09,F1FEE4DB985D4CB7E044001517F43A86,1,9,1,9,1,0,0,0,0,81A6DC7DD1753355A51217D7C3CD787E,IPRR0256974,2024-04-10,276080633,Counterfeit,2024-04-10,1


In [139]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_catalog_table_in_customer""")

In [140]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_catalog_table_in_customer(
order_placed_dt DATE, order_number BIGINT, total_revenue BIGINT, Catalog_totals_orders BIGINT, Catalog_Total_Revenue DOUBLE,
Order_Basis_Points_YTD DOUBLE, Revenue_Basis_Points_YTD DOUBLE, label VARCHAR)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

###import tns_catalog_new table,then combine two tables together#
rate4 = pd.merge(tns_catalog_new_all, CF_rev_30days4, how = 'left', left_on = 'order_plcd_dt', right_on = 'ORDER_PLACED_DT')
rate4.fillna(0, inplace = True)
cumulative_rate4 = rate4[rate4['order_plcd_dt'] >= dt_start]
cumulative_rate4.fillna(0, inplace = True)
cumulative_rate4 = cumulative_rate4.sort_values('order_plcd_dt')
cumulative_rate4['cumulative_Total_Quantity_y'] = cumulative_rate4['Total_Quantity_y'].cumsum()
cumulative_rate4['cumulative_Order_Number'] = cumulative_rate4['Order_Number'].cumsum()
cumulative_rate4['cumulative_Sum(TOT_GMV)'] = cumulative_rate4['Sum(TOT_GMV)'].cumsum()
cumulative_rate4['cumulative_Total_Revenue'] = cumulative_rate4['Total_Revenue'].cumsum()


In [141]:
cumulative_rate4['cumulative_order'] = cumulative_rate4['cumulative_Total_Quantity_y'] / cumulative_rate4['cumulative_Order_Number'] * 10000
cumulative_rate4['cumulative_revenue'] = cumulative_rate4['cumulative_Sum(TOT_GMV)'].astype(float) / cumulative_rate4['cumulative_Total_Revenue'] * 10000

cumulative_rate_sheet4 = cumulative_rate4[['order_plcd_dt', 'Total_Quantity_y', 'Sum(TOT_GMV)', 'Order_Number', 'Total_Revenue', 'cumulative_order', 'cumulative_revenue']]
cumulative_rate_sheet4.rename(columns={'order_plcd_dt':'day',
                   'Total_Quantity_y':'Counterfeits Orders',
                   'Sum(TOT_GMV)':'Counterfeits Revenue','Order_Number':'Catalog Totals Orders','Total_Revenue':'Catalog Totals Revenue',
                   'cumulative_order':'Order Basis Points(YTD)',
                   'cumulative_revenue':'Revenue Basis Points(YTD)'}, inplace = True)

tns_catalog_new4 = cumulative_rate_sheet4

tns_catalog_new4["label"]="CF Rate(Customer Report)"
tns_catalog_new4.to_csv('catalog_order' +str(date.today())+'_Customer.txt',sep='|',index=False,header=False)

/libs/project/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [142]:
filepath = 'catalog_order' +str(date.today())+'_Customer.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_catalog_table_in_customer/" + filepath)
customer_report = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_catalog_table_in_customer""")
customer_report

,order_placed_dt,order_number,total_revenue,catalog_totals_orders,catalog_total_revenue,order_basis_points_ytd,revenue_basis_points_ytd,label
0,2024-01-27,35,732,368974,1.723096e+07,0.948576,0.424881,CF Rate(Customer Report)
1,2024-01-28,67,2201,393037,1.828005e+07,1.338563,0.826152,CF Rate(Customer Report)
2,2024-01-29,37,1579,363658,1.740815e+07,1.234821,0.852918,CF Rate(Customer Report)
3,2024-01-30,34,1310,376463,1.822693e+07,1.151696,0.818603,CF Rate(Customer Report)
4,2024-01-31,70,3174,395749,1.931757e+07,1.280375,0.994704,CF Rate(Customer Report)
...,...,...,...,...,...,...,...,...
81,2024-04-17,0,0,441744,2.445377e+07,0.625166,0.371454,CF Rate(Customer Report)
82,2024-04-18,0,0,436144,2.409728e+07,0.617301,0.366326,CF Rate(Customer Report)
83,2024-04-19,0,0,419693,2.366817e+07,0.609917,0.361425,CF Rate(Customer Report)
84,2024-04-20,0,0,413370,2.279413e+07,0.602815,0.356827,CF Rate(Customer Report)


### Low risk

In [143]:
claim_table5 = claim_table1[~claim_table1['Claim ID'].isin(excluded_claims_set)]
#claim_table5 = claim_table5[claim_table5['Actual Claim Type'] == 'Counterfeit' & claim_table5(claim_table5['Reported Details'].str.contains('CUSTOMER REPORT')==False)&(claim_table5['IP Company Name'].str.contains('CUSTOMER REPORT')==False)]
claim_table5 = claim_table5[(claim_table5['Actual Claim Type'] == 'Counterfeit') & (claim_table5['Reported Details'].str.contains('CUSTOMER REPORT')==False)&(claim_table5['IP Company Name'].str.contains('CUSTOMER REPORT')==False)]

In [144]:
tns_claim_order5=pd.merge(tns_claim_order, claim_table5.drop_duplicates(subset = 'ID'), left_on = 'offer_id', right_on = 'ID')
tns_claim_order5=tns_claim_order5[['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number',
       'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity',
       'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount',
       'Return_Quantity']]

In [145]:
Claim_df5 = claim_table5.groupby('ID').agg({'Claim ID' : 'first', 'Created' :'first', 'Item ID' :'first', 'Actual Claim Type' : 'first'})
Claim_df5 = Claim_df5.reset_index()

new_df5 = pd.merge(tns_claim_order5, Claim_df5, left_on = 'offer_id', right_on = 'ID', how = 'left')
new_df5['Claim date']= pd.to_datetime(new_df5['Created']).dt.date
new_df5['order_plcd_dt']= pd.to_datetime(new_df5['order_plcd_dt']).dt.date
new_df5['delta'] = (new_df5['Claim date'] - new_df5['order_plcd_dt']).dt.days
new_df_5 = new_df5[(new_df5['delta'] >= 0) & (new_df5['delta'] <= 30)]
new_df_5.columns = ['offer_id', 'order_plcd_dt', 'cust_id', 'Order_Number', 'Total_Revenue', 'Total_Quantity', 'Placed_Amount', 'Placed_Quantity', 'Total_cancel_Amount', 'Cancelled_Quantity', 'Refunded_Amount', 'Return_Quantity', 'ID', 'Item ID', 'Claim ID', 'Created', 'Actual Claim Type', 'Claim date', 'delta']

CF_rev_30days5 = new_df_5.groupby('order_plcd_dt').agg({'Total_Quantity' :'sum', 'Total_Revenue' : 'sum'})
CF_rev_30days5.reset_index(inplace = True)
CF_rev_30days5=CF_rev_30days5.rename(columns={'order_plcd_dt':'ORDER_PLACED_DT', 'Total_Revenue' : 'Sum(TOT_GMV)'})
CF_rev_30days5['ORDER_PLACED_DT'] = pd.to_datetime(CF_rev_30days5['ORDER_PLACED_DT'])

connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_items_orders_low_risk""")
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_items_orders_low_risk(
offer_id VARCHAR, order_placed_dt DATE, cust_id VARCHAR, order_number BIGINT, total_revenue BIGINT, total_quantity BIGINT,
placed_amount BIGINT, placed_quantity BIGINT,
total_cancel_amount BIGINT, cancelled_quantity BIGINT,
refunded_amount BIGINT, return_quantity BIGINT,
ID VARCHAR, Claim_ID VARCHAR, Created DATE, Item_ID VARCHAR, Actual_Claim_Type VARCHAR, Claim_date DATE, delta BIGINT)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

new_df_5.to_csv('item_order' +str(date.today())+'_low_risk.txt',sep='|',index=False,header=False)
filepath = 'item_order' +str(date.today())+'_low_risk.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_items_orders_low_risk/" + filepath)
tns_claims_items_orders5 = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_items_orders_low_risk""")

connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_catalog_table_counterfeit""")
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_catalog_table_counterfeit(
order_placed_dt DATE, order_number BIGINT, total_revenue BIGINT, Catalog_totals_orders BIGINT, Catalog_Total_Revenue DOUBLE,
Order_Basis_Points_YTD DOUBLE, Revenue_Basis_Points_YTD DOUBLE, label VARCHAR)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

###import tns_catalog_new table,then combine two tables together#
rate5 = pd.merge(tns_catalog_new_all, CF_rev_30days5, how = 'left', left_on = 'order_plcd_dt', right_on = 'ORDER_PLACED_DT')
rate5.fillna(0, inplace = True)
cumulative_rate5 = rate5[rate5['order_plcd_dt'] >= dt_start]
cumulative_rate5.fillna(0, inplace = True)
cumulative_rate5 = cumulative_rate5.sort_values('order_plcd_dt')
cumulative_rate5['cumulative_Total_Quantity_y'] = cumulative_rate5['Total_Quantity_y'].cumsum()
cumulative_rate5['cumulative_Order_Number'] = cumulative_rate5['Order_Number'].cumsum()
cumulative_rate5['cumulative_Sum(TOT_GMV)'] = cumulative_rate5['Sum(TOT_GMV)'].cumsum()
cumulative_rate5['cumulative_Total_Revenue'] = cumulative_rate5['Total_Revenue'].cumsum()
cumulative_rate5['cumulative_order'] = cumulative_rate5['cumulative_Total_Quantity_y'] / cumulative_rate3['cumulative_Order_Number'] * 10000
cumulative_rate5['cumulative_revenue'] = cumulative_rate5['cumulative_Sum(TOT_GMV)'].astype(float) / cumulative_rate3['cumulative_Total_Revenue'] * 10000

cumulative_rate_sheet5 = cumulative_rate5[['order_plcd_dt', 'Total_Quantity_y', 'Sum(TOT_GMV)', 'Order_Number', 'Total_Revenue', 'cumulative_order', 'cumulative_revenue']]
cumulative_rate_sheet5.rename(columns={'order_plcd_dt':'day',
                   'Total_Quantity_y':'Counterfeits Orders',
                   'Sum(TOT_GMV)':'Counterfeits Revenue','Order_Number':'Catalog Totals Orders','Total_Revenue':'Catalog Totals Revenue',
                   'cumulative_order':'Order Basis Points(YTD)',
                   'cumulative_revenue':'Revenue Basis Points(YTD)'}, inplace = True)

tns_catalog_new5 = cumulative_rate_sheet5

tns_catalog_new5["label"]="CF Rate(Low Risk)"
tns_catalog_new5.to_csv('catalog_order' +str(date.today())+'_counterfeit.txt',sep='|',index=False,header=False)
filepath = 'catalog_order' +str(date.today())+'_counterfeit.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_catalog_table_counterfeit/" + filepath)

/libs/project/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [146]:
new_df_5[new_df_5['offer_id']=='D34C22F70606471FB5BEFA0266AB7165']

,offer_id,order_plcd_dt,cust_id,Order_Number,Total_Revenue,Total_Quantity,Placed_Amount,Placed_Quantity,Total_cancel_Amount,Cancelled_Quantity,Refunded_Amount,Return_Quantity,ID,Item ID,Claim ID,Created,Actual Claim Type,Claim date,delta


In [147]:
low_risk = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_catalog_table_counterfeit""")
low_risk

,order_placed_dt,order_number,total_revenue,catalog_totals_orders,catalog_total_revenue,order_basis_points_ytd,revenue_basis_points_ytd,label
0,2024-01-27,28,535,368974,1.723096e+07,0.758861,0.310743,CF Rate(Low Risk)
1,2024-01-28,26,496,393037,1.828005e+07,0.708651,0.290521,CF Rate(Low Risk)
2,2024-01-29,73,1350,363658,1.740815e+07,1.128218,0.450226,CF Rate(Low Risk)
3,2024-01-30,21,414,376463,1.822693e+07,0.985266,0.393088,CF Rate(Low Risk)
4,2024-01-31,2,24,395749,1.931757e+07,0.790355,0.311872,CF Rate(Low Risk)
...,...,...,...,...,...,...,...,...
81,2024-04-17,0,0,441744,2.445377e+07,0.184044,0.074783,CF Rate(Low Risk)
82,2024-04-18,0,0,436144,2.409728e+07,0.181729,0.073751,CF Rate(Low Risk)
83,2024-04-19,0,0,419693,2.366817e+07,0.179555,0.072764,CF Rate(Low Risk)
84,2024-04-20,0,0,413370,2.279413e+07,0.177464,0.071839,CF Rate(Low Risk)


### Combine three tables into one table

In [148]:
#Claim_items_orders#
new_df_1["label"]="CF Rate(All Claims)"
new_df_2["label"]="CF Rate(Rights Owner)"
new_df_3["label"]="CF Rate(High Risk)"
new_df_4["label"]="CF Rate(Customer Report)"
new_df_5["label"]="CF Rate(Low Risk)"
claim_items_orders_table=pd.concat([new_df_1,new_df_2,new_df_3,new_df_4,new_df_5], axis=0).reset_index(drop=True)
claim_items_orders_table

/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,offer_id,order_plcd_dt,cust_id,Order_Number,Total_Revenue,Total_Quantity,Placed_Amount,Placed_Quantity,Total_cancel_Amount,Cancelled_Quantity,Refunded_Amount,Return_Quantity,ID,Item ID,Claim ID,Created,Actual Claim Type,Claim date,delta,label
0,81A6DC7DD1753355A51217D7C3CD787E,2024-03-11,FB02BB801BD64DE28C441C3809F6A57A,1,13.96,2,13.96,2,0,0.0,0,0,81A6DC7DD1753355A51217D7C3CD787E,IPRR0256974,2024-04-10,276080633,Counterfeit,2024-04-10,30,CF Rate(All Claims)
1,36F8C68C50B93B0B91B6CC11B7AA4980,2024-04-01,8BE238773EDA4C119A7C4218961BA40F,1,98.7,1,98.7,1,0,0.0,0,0,36F8C68C50B93B0B91B6CC11B7AA4980,IPRR0256620,2024-04-09,431261416,Confirmed Counterfeit,2024-04-09,8,CF Rate(All Claims)
2,680790AF80E84BEF8AB1944CAA0093BC,2024-04-09,C88D6B49F0D643FC889913C67F84E9C8,1,36.98,1,36.98,1,0,0.0,0,0,680790AF80E84BEF8AB1944CAA0093BC,IPRR0259166,2024-04-18,822221942,Confirmed Counterfeit,2024-04-18,9,CF Rate(All Claims)
3,FC190E57862A3409AF432ABAD725F386,2024-02-20,901F6B2A104241009A83D36F45905B75,1,17.98,1,17.98,1,0,0.0,0,0,FC190E57862A3409AF432ABAD725F386,IPRR0245869,2024-02-21,1191181336,Counterfeit,2024-02-21,1,CF Rate(All Claims)
4,66EB9FE4295831B68A5D009A87582554,2024-04-19,E1DBBC1B51D54D9EAF486EEFEBBBF629,1,25.88,1,25.88,1,0,0.0,0,0,66EB9FE4295831B68A5D009A87582554,IPRR0259439,2024-04-19,3235157718,Confirmed Counterfeit,2024-04-19,0,CF Rate(All Claims)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15318,99FEB3DF0BBA4C89AAD77A84E6B152AD,2024-03-10,FC5736EE5722457DB8A51EBA2871C6DF,1,11.99,1,11.99,1,0,0.0,0,0,99FEB3DF0BBA4C89AAD77A84E6B152AD,IPRR0250771,2024-03-19,3299591079,Counterfeit,2024-03-19,9,CF Rate(Low Risk)
15319,99FEB3DF0BBA4C89AAD77A84E6B152AD,2024-02-20,E06F40232A024F139369A4ABA42063DA,1,11.99,1,11.99,1,0,0.0,0,0,99FEB3DF0BBA4C89AAD77A84E6B152AD,IPRR0250771,2024-03-19,3299591079,Counterfeit,2024-03-19,28,CF Rate(Low Risk)
15320,821687760C123622BA56E6C11A08DD7A,2024-04-08,36BE1B3C61704A49AEB38F44D8B9DE37,1,119.99,1,119.99,1,0,0.0,0,0,821687760C123622BA56E6C11A08DD7A,IPRR0256972,2024-04-10,779130375,Counterfeit,2024-04-10,2,CF Rate(Low Risk)
15321,540B933D1C4937ED836A1A5CF080CBE5,2024-02-13,81D247BB50534380BB219A13C94C122E,1,18.99,1,18.99,1,0,0.0,0,0,540B933D1C4937ED836A1A5CF080CBE5,IPRR0246430,2024-02-26,3809468129,Counterfeit,2024-02-26,13,CF Rate(Low Risk)


In [149]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_items_orders""")

In [150]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_items_orders(
offer_id VARCHAR, order_placed_dt DATE, cust_id VARCHAR, order_number BIGINT, total_revenue BIGINT, total_quantity BIGINT,
placed_amount BIGINT, placed_quantity BIGINT, total_cancel_amount BIGINT, cancelled_quantity BIGINT,
refunded_amount BIGINT, return_quantity BIGINT,
ID VARCHAR, Claim_ID VARCHAR, Created DATE, Item_ID VARCHAR, Actual_Claim_Type VARCHAR, Claim_date DATE, delta BIGINT, label VARCHAR)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

claim_items_orders_table.to_csv('item_order' +str(date.today())+'_all.txt',sep='|',index=False,header=False)
filepath = 'item_order' +str(date.today())+'_all.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_items_orders/" + filepath)

#Claims_orders
claims_items_orders = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_items_orders""")

In [151]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_items_orders_1""")

DatabaseError: java.sql.SQLException: Query failed (#20240422_161216_37932_p75m2): line 1:1: Table 'hive.tnsanalytics.tns_claims_items_orders_1' does not exist

In [ ]:
connector.execute(name="gcp_dd_connection", statement="""create table tnsanalytics.tns_claims_items_orders_1 as
SELECT distinct "tns_claims_items_o"."offer_id" AS "offer_id",
  "tns_claims_items_o"."order_placed_dt" AS "order_placed_dt",
  "tns_claims_items_o"."cust_id" AS "cust_id",
  "tns_claims_items_o"."order_number" AS "order_number",
  "tns_claims_items_o"."total_revenue" AS "total_revenue",
  "tns_claims_items_o"."total_quantity" AS "total_quantity",
  "tns_claims_items_o"."placed_amount" AS "placed_amount",
  "tns_claims_items_o"."placed_quantity" AS "placed_quantity",
  "tns_claims_items_o"."total_cancel_amount" AS "total_cancel_amount",
  "tns_claims_items_o"."cancelled_quantity" AS "cancelled_quantity",
  "tns_claims_items_o"."refunded_amount" AS "refunded_amount",
  "tns_claims_items_o"."return_quantity" AS "return_quantity",
  "tns_claims_items_o"."id" AS "id",
  "tns_claims_items_o"."claim_id" AS "claim_id",
  "tns_claims_items_o"."created" AS "created",
  "tns_claims_items_o"."item_id" AS "item_id",
  "tns_claims_items_o"."actual_claim_type" AS "actual_claim_type",
  "tns_claims_items_o"."claim_date" AS "claim_date",
  "tns_claims_items_o"."delta" AS "delta",
  "tns_claims_items_o"."label" AS "label",
  t.brand_nm as brand_name, t.rpt_lvl_1_orig_nm as category, t.slr_org_cd as seller_id, t.prod_nm as item_name, t.slr_org_nm as seller_name
FROM "tnsanalytics"."tns_claims_items_orders" "tns_claims_items_o"
left join (select brand_nm , cust_id,order_plcd_dt,offr_id,rpt_lvl_1_orig_nm , slr_org_cd , prod_nm , slr_org_nm from ww_crew_dl_rpt_vm.cnsld_order_item
where order_plcd_dt >= date'2024-01-27') t
on "tns_claims_items_o"."cust_id" = t.cust_id and "tns_claims_items_o"."order_placed_dt" = t.order_plcd_dt and "tns_claims_items_o"."offer_id" = t.offr_id
where "tns_claims_items_o"."claim_date" >= date'2024-01-27' """)

In [ ]:
catalog_hist_new=pd.concat([rights_owner,high_risk,low_risk], axis=0).reset_index(drop=True)
catalog_hist_new

In [ ]:
catalog_hist_new.columns

In [ ]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_catalog_table_new""")

In [ ]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_catalog_table_new(
order_placed_dt DATE, order_number BIGINT, total_revenue BIGINT, Catalog_totals_orders BIGINT,Catalog_Total_Revenue DOUBLE,
Order_Basis_Points_YTD DOUBLE, Revenue_Basis_Points_YTD DOUBLE, label VARCHAR)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)
catalog_hist_new.to_csv('catalog_order' +str(date.today())+'_new.txt',sep='|',index=False,header=False)
filepath = 'catalog_order' +str(date.today())+'_new.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_catalog_table_new/" + filepath)

In [ ]:
catalog_hist_new[catalog_hist_new['order_placed_dt']=='2023-02-05']

In [ ]:
query = "select claim_id from tnsanalytics.tns_claims_items_orders group by claim_id "
test  = dataset.load(name="gcp_dd_connection", query = query)
print(test)

In [ ]:
a = cumulative_rate4['cumulative_Total_Quantity_y']+cumulative_rate2['cumulative_Total_Quantity_y']
print(a)

In [ ]:
b= cumulative_rate['cumulative_Total_Quantity_y']
print(b)

In [ ]:
tns_claim_order.to_excel('all.xlsx')

In [ ]:
b= tns_claim_order['Total_Quantity'].fillna(0)
print(b)

In [ ]:
cf_claims_and_order.columns

In [ ]:
cf_claims_and_order.columns = ['ID', 'claim_id', 'created_date', 'item_id', 'actual_claim_type', 'claim_state' ,'action', 'reason_code', 'category', 'offer_id', 'order_plcd_dt', 'order_number', 'total_revenue', 'total_quantity', 'placed_amount', 'placed_quantity', 'total_cancel_amount', 'cancelled_quantity', 'refunded_amount' , 'return_quantity']

In [ ]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.tns_claims_items_orders_cf""")

In [ ]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.tns_claims_items_orders_cf(ID VARCHAR, claim_id VARCHAR,
created_date DATE, item_id VARCHAR,  actual_claim_type VARCHAR, claim_state VARCHAR, action VARCHAR, reason_code VARCHAR, category VARCHAR,
offer_id VARCHAR, order_placed_dt DATE, order_number BIGINT, total_revenue BIGINT, total_quantity BIGINT,
placed_amount BIGINT, placed_quantity BIGINT, total_cancel_amount BIGINT, cancelled_quantity BIGINT, refunded_amount BIGINT, return_quantity BIGINT)
WITH (format = 'TEXTFILE', textfile_field_separator ='|') """)

In [ ]:
#import os
#from pathlib import Path
#from typing import Optional
#from datetime import date

#def save_df(df:pd.DataFrame, file_name: str, file_path: Optional[str] = "data") -> None:
 #   """
 #   Function to save a dataframe to a local file_path with a file_name that is appended with today's date.
 #   Arguments:
 #       df: DataFrame to be saved
 #       file_name: Name of the file to which the dataframe is saved
 #       file_path: File path or folder name where the file is saved. This is optional and defaults to a folder "data" 
 #                  withih your current working directory.
 #   """
 #   Path(file_path).mkdir(parents=True, exist_ok=True)
 #   current_date = date.today()
 #   data_file_path = Path(file_path + f"/{file_name}_{current_date}.parquet")
 #   
 #   if not data_file_path.exists():
 #       df.to_parquet(data_file_path)
 #       print(f"Dataframe saved as {file_name}_{current_date}.parquet in the folder:{file_path}")
 #   else:
 #       print(f"File {file_name}_{current_date}.parquet already exists in the folder:{file_path}")

In [ ]:
cf_claims_and_order.to_csv('CF_item_order' +str(date.today())+'_all.txt',sep='|',index=False,header=False)
filepath = 'CF_item_order' +str(date.today())+'_all.txt'
connector.upload(name="gcs_connector", container="presto-tables", source_path=filepath, target_path="tns_claims_items_orders_cf/" + filepath)

#Claims_orders
CF_items_orders = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_items_orders_cf""")

In [ ]:
save_df(df=cf_claims_and_order.head(), file_name="cf_claims_and_order")

In [ ]:
CF_items_orders.tail()

In [ ]:
claim_table['ID'].nunique()

In [ ]:
CF_items_orders.columns

In [ ]:
CF_items_orders['offer_id'].nunique()

In [ ]:
def delete_files(file_path: str) -> None:
    """
    Function to delete all files from a given file_path.
    Arguments:
        file_path: File path or folder name from where all the existing files will be deleted. 
    """
    file_path = Path(file_path)
    files = os.listdir(file_path)
    
    for file in files:
        data_file_path = os.path.join(file_path, file)
        if Path(data_file_path).exists():
            os.remove(data_file_path)
            print(f"{data_file_path} has been deleted from folder:{file_path}.")
        else:
            print(f"{data_file_path} does not exist.")

In [ ]:
delete_files(file_path="data")

In [ ]:
connector.upload(name="gcs_connector", container="presto-tables", source_path="data/cf_claims_and_order_2024-04-02.parquet", target_path="tns_claims_items_orders_cf_new/" + filepath)

In [ ]:
CF_items_orders = dataset.load(name="gcp_dd_connection", query="""select * from tnsanalytics.tns_claims_items_orders_cf_new""")

In [ ]:
CF_items_orders.shape

In [ ]:
temp_df = CF_items_orders.head()
temp_df

In [ ]:
temp_df.to_parquet("data/test_df.parquet")

In [ ]:
df = pd.read_parquet("data/test_df.parquet")

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
connector.execute(name="gcp_dd_connection", statement="""DROP TABLE tnsanalytics.test_sravan""")

In [ ]:
connector.execute(name="gcp_dd_connection", statement="""SET SESSION hive.parquet_use_column_names = true""")

In [ ]:
connector.execute(name="gcp_dd_connection", statement="""CREATE TABLE tnsanalytics.test_sravan 
(ID VARCHAR, 
claim_id VARCHAR,
created_date DATE, 
item_id VARCHAR,  
actual_claim_type VARCHAR, 
claim_state VARCHAR, 
action VARCHAR, 
reason_code VARCHAR, 
category VARCHAR,
offer_id VARCHAR, 
order_placed_dt DATE, 
order_number BIGINT, 
total_revenue BIGINT, 
total_quantity BIGINT,
placed_amount BIGINT, 
placed_quantity BIGINT, 
total_cancel_amount BIGINT, 
cancelled_quantity BIGINT, 
refunded_amount BIGINT, 
return_quantity BIGINT)
WITH (format = 'parquet')""")

In [ ]:
connector.upload(name="gcs_connector", 
                 container="presto-tables", 
                 source_path="data/test_df.parquet", 
                 target_path="test_sravan/" + "test_df.parquet")

